# 네이버 블로그로 파급력을 측정합니다.

<naverapi 응답 구조>  
lastBuildDate : api 요청한 시각  
total : 관련 블로그 수  
start : 시작  
display : 가져온 갯수  
items : 블로그  
<items 구조>  
title : 블로그 제목    
link : 블로그 링크  
description : 블로그 내용  
bloggername : 블로그 이름  
bloggerlink : 블로그 메인화면  
postdate : 게시날짜  

In [ ]:
import urllib.request
import json
import pandas as pd
import re
from konlpy.tag import Kkma

In [ ]:
class NaverData:
    count = -1
    power = pd.DataFrame(columns = ["id","resource", "restaurant", "before", "after", "blogTotal", "blogDisplay", "referenceDate"])
    power.set_index(['id'], inplace=True)
    findData = pd.DataFrame(columns = ["id", "resource", "restaurant", "data"])
    findData.set_index(['id'], inplace=True)
    
    def __init__(self, resource, restaurant, display=20, referenceDate=None):
        self.resource = resource
        self.restaurant = restaurant
        naverData = NaverData.naverBlog(restaurant, display)
        self.lastBuildDate = naverData["lastBuildDate"]
        self.total = naverData["total"]
        self.start = naverData["start"]
        self.display = naverData["display"]
        self.items = NaverData.cleanIteams(pd.DataFrame(naverData["items"]))
        self.referenceData = self.findResource()
        self.referenceDate = referenceDate
        NaverData.count = NaverData.count + 1
        self.id = NaverData.count
        self.addPower()
        NaverData.findData.loc[self.id] = [resource, restaurant, self]
    
    def __del__(self):
        NaverData.power.drop(self.id, inplace=True)
        NaverData.findData.drop(self.id, inplace=True)
        print(self.id, "번 데이터를 지웁니다.")
    
    # 네이버 ReatAPI
    def naverBlog(query, num) :
        client_key = 'izGsqP2exeThwwEUVU3x'
        client_secret = 'WrwbQ1l6ZI'
        encText = urllib.parse.quote_plus(query)
        naver_url = 'https://openapi.naver.com/v1/search/blog.json?query=' + encText + '&display=' + str(num)
        print("url:", naver_url)

        # request 객체에 add하기
        request = urllib.request.Request(naver_url)
        request.add_header("X-Naver-Client-Id",client_key)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        # 응답받은 코드가 정상적인지 확인하기
        rescode = response.getcode()
        if(rescode == 200):
            response_body = response.read()
            dataList = json.loads(response_body)
            return dataList
        else:
            print('오류 코드 : ' + rescode)
    
    def cleanIteams(itemsdf):
        """
        items데이터프라임 -> items데이터프라임
         - items데이터프라임 title열과 description열에서 <b></b>&quot; 등 없얘기
         - items데이터프레임 postdate 타입을 datetime으로 변경하기
        """
        for i in range(itemsdf.shape[0]):
            # title열 데이터 정리하기
            title = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"title"]))
            title = re.sub("&.{1,5};", "", title)
            title = re.sub(" ( )+", "", title)
            itemsdf.loc[i,"title"] = title
            # description열 데이터 정리하기
            description = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"description"]))
            description = re.sub("&.{1,5};", "", description)
            description = re.sub(" ( )+", "", description)
            itemsdf.loc[i,"description"] = description
            # postdate열 타입을 datetime로 바꾸기
            itemsdf['postdate'] = pd.to_datetime(itemsdf['postdate'])
        return itemsdf
    
    def findResource(self):
        """
        items 데이터프레임 title열과 description열에서 resource단어가 들어간 데이터를 찾아 줍니다.
        """
        kkma = Kkma()
        mask = (self.items["title"] + self.items["description"]).map(kkma.nouns).map(lambda x: self.resource in x)
        return self.items.loc[mask]
    
    def findOldestDate(self):
        """
        찾아온 데이터 중 블로그 제목과 설명에 제공자이름이 들어간 데이터 중 가장 오래된 데이터를 찾습니다.
        """
        if len(self.referenceData) <= 0:
            print("해당 게시물이 없습니다.")
            return "2099-12-31 00:00:00"
        else:
            return self.referenceData.sort_values(by='postdate', ascending=True).iloc[0]["postdate"]
    
    def countBF(self):
        """
        기준일을 기준으로 이전 게시물 갯수와 이후 게시물 갯수를 세줍니다.
        반환값 : {'before':X, 'after': X}
        """
        # 기준일을 받지 않으면 게시물 중 가장 오래된 날짜로 지정한다.
        if self.referenceDate is None:
            self.referenceDate = self.findOldestDate()
        before = len(self.referenceData.loc[self.referenceData['postdate'] < self.referenceDate])
        after = len(self.referenceData.loc[self.referenceData['postdate'] >= self.referenceDate])
        return {'before':before, 'after': after}
    
    def addPower(self):
        bf = self.countBF()
        NaverData.power.loc[self.id] = [self.resource, self.restaurant, bf["before"],
                                        bf["after"], self.total, self.display, self.referenceDate]
        display(NaverData.power)

In [ ]:
naverdata1 = NaverData("츄릅켠", "소문난원조쌈밥")
naverdata2 = NaverData("김준현", "원조수구레")

In [ ]:
del naverdata2

In [ ]:
# 클래스 메소들 만들기
mask = (NaverData.findData["resource"] == "김준현") & (NaverData.findData["restaurant"] == "원조수구레")
NaverData.findData.loc[mask].iloc[0]["data"].items

# 실험

In [ ]:
# html 빼기
text = "민락<동 얼큰칼국수&quot;<b>츄릅켠</b> 님 방송 보고 다녀옴!!!&quot;"
text = re.sub("<b>|</b>", " ", text)
text = re.sub("&.{1,5};", "", text)
print(text)

In [ ]:
sample = "해운대 원조할매국밥을 찾아가듯이 이 분위기와 쌈밥이 땡길때면 재방문할 의향이 있네요..."
kkma = Kkma() 
kkma.nouns(sample)
# pprint(kkma.morphs(sample))
# pprint(kkma.pos(sample))

# 문제모음

1. 인스턴스가 삭제가 안됩니다.